In [3]:
from equations import *

In [4]:
altitude_cr = 11887.2 # m, 39000 ft, From REQ-CRUISE-01
isa(altitude_cr)

(216.6, 19648.295168950015, 0.31607090848028313)

In [5]:
V_cr

241.90678227780217

In [3]:
W_S = np.arange(0, 9000, step=100)
W_S

In [5]:
W_S_min_speed = min_speed()
W_S_min_speed

In [6]:
W_S_landing_field_length = landing_field(h_landing=0)
W_S_landing_field_length

In [8]:
W_S_climb_rate = climb_rate(W_S[4:])
# W_S_climb_rate

In [11]:
df = pd.read_csv(sheets_url, header=[0,1], nrows=8)

In [12]:
df

In [14]:
fig, ax = plt.subplots(figsize=(14, 9), layout='constrained')

plt.plot(W_S[4:], W_S_takeoff_field, label="REQ-OFF-01 Take-off Field Length", color="green")
plt.plot(W_S, W_S_climb_gradient_25_121a, label="REQ-OFF-02 Climb Gradient Take-off")
plt.plot(W_S, W_S_climb_gradient_25_121b, label="REQ-OFF-03 Climb Gradient Take-off")
plt.plot(W_S, W_S_climb_gradient_25_121c, label="REQ-OFF-04 Climb Gradient Take-off")

plt.plot(W_S[4:], W_S_climb_rate, label="REQ-OPER-01 Climb Rate", color="yellow")

plt.plot(W_S[4:], W_S_cruise_speed, label="REQ-CRU-01 Cruise Speed", color="purple")

plt.vlines([W_S_landing_field_length], ymin=0, ymax=1, label="REQ-LAND-01 Landing Field Length", color="orange")
plt.plot(W_S, W_S_climb_gradient_25_119, label="REQ-LAND-02 Climb Gradient Landing")
plt.plot(W_S, W_S_climb_gradient_25_121d, label="REQ-LAND-03 Climb Gradient Approach")
plt.vlines([W_S_min_speed], ymin=0, ymax=1, label="REQ-LAND-05 Minimum Speed", color="b")

plt.scatter(df["W/S"].values, df["T_TO/W_TO"].values, c="blue", s=20, label="Reference Aircraft")
plt.scatter([5472], [0.30431], c="red", s=30, label="Design Point")

plt.ylim((0, 0.6))
plt.xlim((2800, 8400))

ax.xaxis.set_major_locator(plticker.MultipleLocator(400)) 
ax.yaxis.set_major_locator(plticker.MultipleLocator(0.1)) 

plt.grid()

plt.xlabel("W/S [N/m^2]")
plt.ylabel("T/W [N/N]")
plt.legend()
# plt.savefig('matching_diagram.svg', format='svg')
# plt.legend(*p1.legend_elements(), loc=4)
plt.show()

In [25]:
climb_gradient(5472.150325258757, altitude_climb_rate=0, zero_lift_drag_coeff=c_d0(gear_extended=False, flap_deflection=flap_deflection_landing), oswald_factor=oswald_eff(flap_deflection_landing), climb_grad_percent=2.1, oei_condition=True)

In [26]:
S = 1/5472.150325258757 * weight_takeoff
S

In [19]:
l_to

2790

In [21]:
c_d0(gear_extended=False, flap_deflection=0)

0.0156

In [22]:
c_d0(gear_extended=True, flap_deflection=flap_deflection_landing)

0.0811

# Indexing

In [23]:
df.iloc[1]

Manufacturer           Unit                       Boeing
Aircraft name          /                      B777-200LR
Maximum payload        [kg]                        50850
Range                  [km]                        16200
Maximum take-off mass  [kg]                       347800
Operating Empty Mass   [kg]                       156000
Wing Area              [m2]                          437
Wing span              [m]                          64.8
Aspect ratio           [ ]                          9.61
Mach                   [ ]                          0.85
Taper ration           [ ]                         0.149
Sweep angle            [°]                          31.6
Total Take-off thrust  [kN]                          978
Take off distance      [m]                          2900
Landing distance       [m]                          1700
W/S                    [N/m^2]                      7808
T_TO/W_TO              [N/N]                       0.287
Reference(s)           Unnamed:

In [24]:
df.iloc[1]["Range [km]"]

KeyError: 'Range [km]'

In [ ]:
2 + 5

7

# Header!

Markdown

This is fancy code:

In [ ]:
i = 1